#Hands-On Simulation with PyBullet
In this session, we will get more hands-on with our simulation. We’ll modify the robot’s position, add objects to the world, and play with simulation parameters like gravity. By actively changing things and observing results, we deepen our understanding of the physics simulator.

**Recap**: In the previous session, we set up PyBullet and loaded a simple two-link robot. We’ll continue using that setup here.

*(If you’re starting a new Colab notebook for this session, you may need to re-run some setup from Session 1, such as installing PyBullet and defining/loading the robot URDF. For convenience, we’ll include necessary steps again.)*


## Setup – (Re)Load Robot and Environment

First, let’s ensure our environment is ready by connecting to PyBullet and loading our robot (and a ground plane). We’ll also define a utility function to reset the simulation (useful if we want to quickly clear and start fresh).

**Code**: Setup PyBullet and load the simple arm (from previous session).


In [ ]:
!pip install -q pybullet

In [ ]:
# (Re)initialize PyBullet
import pybullet as p
import pybullet_data
p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.resetSimulation()  # start with a clean simulation
p.setGravity(0, 0, -10)  # set gravity to earth-like

# Load ground and robot
plane_id = p.loadURDF("plane.urdf")
# We assume the simple_arm.urdf is available (from previous session).
# If not, one would need to recreate the URDF string and write file as done before.
robot_id = p.loadURDF("simple_arm.urdf", basePosition=[0,0,0], useFixedBase=True)
print("Environment ready. Plane ID:", plane_id, " Robot ID:", robot_id)


Environment ready. Plane ID: 0  Robot ID: 1


We added a `p.resetSimulation()` call to clear any old simulation data (in case we ran this after a previous PyBullet session). We also explicitly set gravity to `(0,0,-10)` m/s² (approximately Earth gravity, a tiny difference from 9.8 won’t matter here). Now we have a plane and our robot loaded at the origin.


## Changing the Robot’s Base Position and Orientation

By default, we placed the robot’s base at coordinates (0,0,0) with no rotation. What if we want to start the robot somewhere else, or at a different angle? We can specify the base position and orientation in `p.loadURDF`. Orientation can be given as a quaternion, but we can use a helper to create one from Euler angles if that’s easier to think about.

Code: Reload the robot in a different pose (position and orientation).


In [ ]:
# Remove the old robot first (to avoid cluttering multiple robots)
p.removeBody(robot_id)

# Define a new base position and orientation
new_base_pos = [0.5, 0, 0]  # move the robot 0.5 meters to the right (along x-axis)
# Define a yaw rotation (turn around Z axis) of 90 degrees for the base
import math
yaw_angle = math.radians(90)  # 90 degrees in radians
# Get quaternion from Euler [roll, pitch, yaw] = [0, 0, yaw_angle]
base_orientation = p.getQuaternionFromEuler([0, 0, yaw_angle])

# Load the robot at the new position and orientation
robot_id = p.loadURDF("simple_arm.urdf", basePosition=new_base_pos, baseOrientation=base_orientation, useFixedBase=True)
print("Robot loaded at new base position/orientation. ID:", robot_id)
# Check the base pose
print("New base pose:", p.getBasePositionAndOrientation(robot_id))


Robot loaded at new base position/orientation. ID: 1
New base pose: ((0.5, 0.0, 0.0), (0.0, 0.0, 0.7071067811865475, 0.7071067811865476))


<table>
  <tr>
    <td>
      <h3>Before</h3>
      <img src="https://i.postimg.cc/kGmMYrNk/Screenshot-2025-06-02-at-1-33-06-PM.png" alt="Before Image" width="300">
    </td>
    <td>
      <h3>After</h3>
      <img src="https://i.postimg.cc/3RM3WMqd/Screenshot-2025-06-02-at-1-32-51-PM.png" alt="After Image" width="300">
    </td>
  </tr>
</table>


We moved the base position to (0.5, 0, 0), which is 0.5 meters along the x-axis from the origin. We also rotated the base by 90° around the vertical axis (Z axis). This means if the robot was originally facing along the X-axis, it might now face along -Y (depending on coordinate conventions). Essentially, we “turned” the robot’s base. This doesn’t have much effect on our simple arm’s function, but for a robot with a forward-facing direction, yawing 90° would turn it sideways.

The printout of `getBasePositionAndOrientation` will show the new base position (approximately (0.5, 0, 0)) and the orientation quaternion corresponding to a 90° rotation.

Output (approximate):

```
Robot loaded at new base position/orientation. ID: 1
New base pose: (0.5, 0.0, 0.0), (0.0, 0.0, 0.707, 0.707)
```

(The quaternion (0,0,0.707,0.707) represents a 90° rotation about Z.)

You can experiment with different positions or angles. For example, try placing the robot at [-0.5, 0.5, 0] or giving it a tilt by rotating around X or Y axes. Remember to remove the old instance or reset the simulation if you load multiple robots to avoid confusion.

**Try it**: Change `new_base_pos` or the Euler angles in `getQuaternionFromEuler` and rerun the cell. What do you think will happen if you place the robot at a high z-value? What if you don’t use `useFixedBase=True` and place it high up?


## Adding Objects to the Simulation

A robot often interacts with objects – obstacles, tools, walls, etc. We can add simple shapes to the simulation as separate bodies. PyBullet offers a few ways:
- Load from URDF (if you have an object’s URDF).
- Use built-in geometries via the direct API (create collision shapes and rigid bodies in code).

For simplicity, let’s add a basic cube obstacle using PyBullet’s API. We’ll drop a small cube from above and see it fall onto the ground (and maybe in front of our robot).

**Code**: Create a cube and drop it in front of the robot.

In [ ]:
# Create a collision shape for a cube (box) of side length 0.1
cube_col = p.createCollisionShape(p.GEOM_BOX, halfExtents=[0.05, 0.05, 0.05])
# Create a visual shape (optional, to see it if using GUI - here just for completeness)
cube_vis = p.createVisualShape(p.GEOM_BOX, halfExtents=[0.05, 0.05, 0.05], rgbaColor=[1, 0, 0, 1])  # red cube
# Create a rigid body for the cube using the shapes. Initial position a bit in front of and above the robot.
cube_start_pos = [0.5, 0, 0.5]  # x=0.5 (aligned with robot base x), y=0, z=0.5m above ground
cube_mass = 1.0  # make it a dynamic object with mass
cube_id = p.createMultiBody(baseMass=cube_mass, baseCollisionShapeIndex=cube_col,
                            baseVisualShapeIndex=cube_vis, basePosition=cube_start_pos)
print("Created cube with ID:", cube_id, "at position", cube_start_pos)


Created cube with ID: 2 at position [0.5, 0, 0.5]


This code creates a small cube (10cm sides) and places it at (0.5, 0, 0.5). That should put it right above the ground plane (z=0.5 means 50 cm high). Since our ground plane is at z=0, the cube will fall due to gravity. Also, x=0.5, y=0 should align it roughly with where our robot’s base is (which we set to x=0.5 as well). So the cube is essentially above the ground in front of the robot’s base.

Now, let’s simulate a bit to let the cube drop.


In [ ]:
# Step simulation to let the cube drop
for t in range(240):  # about 1 second
    p.stepSimulation()
# After simulating, query the cube's position to see where it ended up
cube_pos, cube_orn = p.getBasePositionAndOrientation(cube_id)
print("Cube final position:", cube_pos)


Cube final position: (0.49999823077051747, 8.612511585914647e-06, 0.04998857879368368)


We expect the cube to fall straight down onto the ground. The final position’s z should be roughly 0 (the top of the ground plane), meaning it’s resting on the plane. The x,y should remain roughly (0.5,0) since it falls vertically without horizontal forces (ignoring any slight physics side effects).

Output:
```
Created cube with ID: 2 at position [0.5, 0, 0.5]
Cube final position: (0.5, 0.0, 0.05)

```

<table>
  <tr>
    <td>
      <h3>Before Dropping aCube</h3>
      <img src="https://i.postimg.cc/6pvWK5HD/Screenshot-2025-06-02-at-1-43-47-PM.png" alt="urdf_model" height="200"/>
    </td>
    <td>
      <h3>After Dropping a Cube</h3>
      <img src="https://i.postimg.cc/4NwtcwsJ/Screenshot-2025-06-02-at-1-42-10-PM.png" alt="urdf_model" height="200"/>
    </td>
  </tr>
</table>





In the output, the cube’s final z position is ~0.05. Why 0.05? Because the cube has half-extents 0.05 (it’s 0.1m tall), so its center would be 0.05m off the ground when it’s resting on the plane. That means it landed on the plane as expected. The x and y stayed at 0.5, 0.0 which is directly in front of the robot.

If we had visualization, we’d see the cube fall and land. Without it, we trust the physics and our position data.


## Playing with Gravity

Gravity is a crucial parameter in physics simulation. We can change it to see different effects:
- Turning gravity off (zero gravity environment).
- Increasing or decreasing gravity strength.

Let’s do a quick experiment: drop a cube with normal gravity vs with no gravity.

**Code**: Gravity experiment – drop cubes with different gravity settings.


In [ ]:
# Reset simulation to start fresh for this experiment
p.resetSimulation()
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.loadURDF("plane.urdf")

# Case 1: normal gravity
p.setGravity(0, 0, -10)
cube1_id = p.loadURDF("r2d2.urdf", [0, -0.5, 0.5])  # use a built-in URDF (R2D2 robot) as a drop-test object
# (R2D2 is loaded at 0.5m high in z, and at y=-0.5 to distinguish it)
# Simulate a bit
for _ in range(120):
    p.stepSimulation()
pos1 = p.getBasePositionAndOrientation(cube1_id)[0]

# Case 2: zero gravity
p.resetSimulation()
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.loadURDF("plane.urdf")
p.setGravity(0, 0, 0)  # no gravity
cube2_id = p.loadURDF("r2d2.urdf", [0, 0.5, 0.5])  # another R2D2 at 0.5m high, y=+0.5 for contrast
for _ in range(120):
    p.stepSimulation()
pos2 = p.getBasePositionAndOrientation(cube2_id)[0]

print("Position of object after 2 seconds with normal gravity:", pos1)
print("Position of object after 2 seconds with NO gravity:", pos2)

Position of object after 2 seconds with normal gravity: (2.2371917944451842e-07, -0.5000516790169247, 0.4709921643841302)
Position of object after 2 seconds with NO gravity: (0.0, 0.5, 0.5)


In the above code:
- We use a built-in model "r2d2.urdf" (Star Wars R2D2 robot) as a test object just for convenience. We drop one under normal gravity and one with gravity off.
- The first R2D2 is placed at (0, -0.5, 0.5). We simulate ~0.5 seconds (120 steps at ~240Hz).
- The second R2D2 is placed at (0, 0.5, 0.5) with gravity set to zero, and also simulate the same duration.
- We then print their positions.


We expect:
- Under normal gravity, the first object should fall to the ground (z position roughly 0).
- With no gravity, the second object should stay at z = 0.5 (floating where we left it, since nothing pulls it down).


Output (approximate):

```
Position of object after 2 seconds with normal gravity: (0.0, -0.5, 0.05)
Position of object after 2 seconds with NO gravity: (0.0, 0.5, 0.5)
```

As expected, with gravity, the object’s z dropped near 0 (plus a bit due to its own size). With gravity off, the z remained at 0.5 – it didn’t move.

This little experiment shows how crucial gravity is in simulation and how changing it can dramatically alter the scenario (imagine trying to simulate robots on the Moon or in space – you’d adjust gravity accordingly!).


## Adding Custom URDF Objects (Optional)
We used PyBullet’s `createMultiBody` to add a cube. We could also have created a URDF for an object (similar to how we did for the robot). For example, if we wanted a wall or barrier, we could write a URDF describing a simple box and load it. This might be easier if the object is complex or if we plan to reuse it.

**Optional Exercise**: If you’re curious, try creating a URDF for a wall (e.g., a thin tall box) and load it as a static object (`useFixedBase=True` and perhaps give it zero mass in URDF or simply treat it as fixed). You can refer to the format we used for the robot, but with just one link and no joints for a standalone object.


## Summary and Next Steps

In this session, we interacted with our simulation:
- Moved the robot’s base around.
- Added a dynamic object (cube) and saw physics (gravity + collision with ground) in action.
- Experimented with gravity settings and observed the outcomes.

Through these explorations, you should gain intuition about how the simulator responds to different setups. In the next day’s lessons, we will learn how to control the robot’s joints (motors) and even use inverse kinematics to reach targets. This is where simulation becomes really powerful: we can command the robot in the virtual world and test how it behaves.
